<a href="https://colab.research.google.com/github/nileshgode/Advanced-NLP-Projects-with-TensorFlow-2.0/blob/master/Sarcasm_detection_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sarcasm Detection using Convolutional Neural Networks

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Import the json Data file in drive for read by pandas

In [2]:
import os
import json
os.chdir("/content/drive/My Drive/Colab Notebooks")


In [3]:
import pandas as pd

data = pd.read_json('/content/drive/My Drive/Colab Notebooks/Sarcasm_Headlines_Dataset_v2.json', lines=True)

In [4]:
data

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...
...,...,...,...
28614,1,jews to celebrate rosh hashasha or something,https://www.theonion.com/jews-to-celebrate-ros...
28615,1,internal affairs investigator disappointed con...,https://local.theonion.com/internal-affairs-in...
28616,0,the most beautiful acceptance speech this week...,https://www.huffingtonpost.com/entry/andrew-ah...
28617,1,mars probe destroyed by orbiting spielberg-gat...,https://www.theonion.com/mars-probe-destroyed-...


Import other Necessary libraries

In [6]:
% matplotlib inline
import numpy as np
import re
import gensim
import math
import nltk

from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import KeyedVectors

# supress warnings
import warnings
warnings.filterwarnings('ignore')

In [7]:
def parse_data(file):
    for l in open(file,'r'):
        yield json.loads(l)

data = list(parse_data('Sarcasm_Headlines_Dataset_v2.json'))
df = pd.DataFrame(data)

In [8]:
df.head(5)

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


As Above Data Contain Binary Values given each headline, article_link is not useful for us so we will remove that with .pop command

**Performing basic data analysis and preprocessing our data**

In [9]:
df.pop('article_link')

0        https://www.theonion.com/thirtysomething-scien...
1        https://www.huffingtonpost.com/entry/donna-edw...
2        https://www.huffingtonpost.com/entry/eat-your-...
3        https://local.theonion.com/inclement-weather-p...
4        https://www.theonion.com/mother-comes-pretty-c...
                               ...                        
28614    https://www.theonion.com/jews-to-celebrate-ros...
28615    https://local.theonion.com/internal-affairs-in...
28616    https://www.huffingtonpost.com/entry/andrew-ah...
28617    https://www.theonion.com/mars-probe-destroyed-...
28618    https://www.theonion.com/dad-clarifies-this-no...
Name: article_link, Length: 28619, dtype: object

In [10]:
len(df)

28619

In [11]:
df['headline'][4]

"mother comes pretty close to using word 'streaming' correctly"

In [12]:
df['headline'][1]

'dem rep. totally nails why congress is falling short on gender, racial equality'

In [13]:
classes = np.unique(np.array(df['is_sarcastic']))
classes

array([0, 1])

Performing basic data analysis and preprocessing our data by performing following steps:
1. Remove special characters.
2. Keep only alphanumeric data.
3. Remove stopwords
4. Lemmatize our data
5. Perform case-folding on the data

In [14]:
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords

def text_clean(corpus):
    '''
    Purpose : Function to keep only alphabets, digits and certain words (punctuations, qmarks, tabs etc. removed)
    
    Input : Takes a text corpus, 'corpus' to be cleaned along with a list of words, 'keep_list', which have to be retained
            even after the cleaning process
    
    Output : Returns the cleaned text corpus
    
    '''
    cleaned_corpus = pd.Series()
    for row in corpus:
        qs = []                                                                # Collect Clean text as a series
        for word in row.split():
            p1 = re.sub(pattern='[^a-zA-Z0-9]',repl=' ',string=word)           # With Regx remove any numbers in words
            p1 = p1.lower()                                                    # Convert into Lower Case
            qs.append(p1)                                                      # Collected text append into 'qs'
        cleaned_corpus = cleaned_corpus.append(pd.Series(' '.join(qs)))
    return cleaned_corpus

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [15]:
## Removal of Stopwords if Any 
def stopwords_removal(corpus):
    stop = set(stopwords.words('english'))
    corpus = [[x for x in x.split() if x not in stop] for x in corpus]
    return corpus

In [16]:
## Convert original text into its lematize form 
def lemmatize(corpus):
    lem = WordNetLemmatizer()
    corpus = [[lem.lemmatize(x, pos = 'v') for x in x] for x in corpus]
    return corpus


In [17]:
## Perform Stemming Operation on corpus, Snowball has faster operation than porter stemmer
def stem(corpus, stem_type = None):
    if stem_type == 'snowball':
        stemmer = SnowballStemmer(language = 'english')
        corpus = [[stemmer.stem(x) for x in x] for x in corpus]
    else :
        stemmer = PorterStemmer()
        corpus = [[stemmer.stem(x) for x in x] for x in corpus]
    return corpus

In [18]:
def preprocess(corpus, cleaning = True, stemming = False, stem_type = None, lemmatization = False, remove_stopwords = True):
    
    '''
    Purpose : Function to perform all pre-processing tasks (cleaning, stemming, lemmatization, stopwords removal etc.)
    
    Input : 
    'corpus' - Text corpus on which pre-processing tasks will be performed
    
    'cleaning', 'stemming', 'lemmatization', 'remove_stopwords' - Boolean variables indicating whether a particular task should 
                                                                  be performed or not
    'stem_type' - Choose between Porter stemmer or Snowball(Porter2) stemmer. Default is "None", which corresponds to Porter
                  Stemmer. 'snowball' corresponds to Snowball Stemmer
    
    Note : Either stemming or lemmatization should be used. There's no benefit of using both of them together
    
    Output : Returns the processed text corpus
    
    '''
    if cleaning == True:
        corpus = text_clean(corpus)
    
    if remove_stopwords == True:
        corpus = stopwords_removal(corpus)
    else :
        corpus = [[x for x in x.split()] for x in corpus]
    
    if lemmatization == True:
        corpus = lemmatize(corpus)
        
        
    if stemming == True:
        corpus = stem(corpus, stem_type)
    
    corpus = [' '.join(x) for x in corpus]
        

    return corpus

In [19]:
headlines = preprocess(df['headline'], lemmatization = True, remove_stopwords = True)

In [20]:
headlines[0:5]

['thirtysomething scientists unveil doomsday clock hair loss',
 'dem rep totally nail congress fall short gender racial equality',
 'eat veggies 9 deliciously different recipes',
 'inclement weather prevent liar get work',
 'mother come pretty close use word stream correctly']

In [21]:
## Import the word2vec Model, check for its availability first, if not present dowload it from api and load 'word2vec-google-news-300'
## It will take about 10 minute for first download to load near about 1.5GB data
## Once It will be loaded, we can do further process

from gensim.models import Word2Vec 
import gensim.downloader as api
    
v2w_model=None;
try:
    v2w_model = gensim.models.KeyedVectors.load("./w2vecmodel.mod")
    print("Loaded w2v model")
except:            
    v2w_model = api.load('word2vec-google-news-300')
    v2w_model.save("./w2vecmodel.mod")
    print("Saved w2vec model")

Loaded w2v model


In [22]:
## send the word vectors, each of size 300, to our CNN model.
## If any of the headlines contain more than 10 characters, they will be subsampled to keep the word embeddings for the first 10 characters. 
## If any of the headlines contain less than 10 characters, we will pad them so that they have vectors with values of 0.

MAX_LENGTH = 10
VECTOR_SIZE = 300

In [23]:
## 1. convert the preprocessed headlines into their respective vectors
## 2. pad or subsample the data based on whether the size of the data is greater or less than the maximum length we have defined

def vectorize_data(data):
    
    vectors = []
    
    padding_vector = [0.0] * VECTOR_SIZE
    
    for i, data_point in enumerate(data):
        data_point_vectors = []
        count = 0
        
        tokens = data_point.split()
        
        for token in tokens:
            if count >= MAX_LENGTH:
                break
            if token in v2w_model.wv.vocab:
                data_point_vectors.append(v2w_model.wv[token])
            count = count + 1
        
        if len(data_point_vectors) < MAX_LENGTH:
            to_fill = MAX_LENGTH - len(data_point_vectors)
            for _ in range(to_fill):
                data_point_vectors.append(padding_vector)
        
        vectors.append(data_point_vectors)
        
    return vectors

In [24]:
## The 'vectorized_headlines' parameter contains our headlines after they've been converted into vectors using Word2Vec. 

vectorized_headlines = vectorize_data(headlines)

In [25]:
## Add a small validation to ensure that the 10 vectors are present for each headline
for i, vec in enumerate(vectorized_headlines):
    if len(vec) != MAX_LENGTH:
        print(i)

In [26]:
len(vectorized_headlines[1])

10

In [27]:
len(vectorized_headlines)

28619

In [28]:
## Select Data train:test ratio as 70:30
train_div = math.floor(0.7 * len(vectorized_headlines))
train_div

20033

In [29]:
## split our data into train and test sets

X_train = vectorized_headlines[:train_div]
y_train = df['is_sarcastic'][:train_div]
X_test = vectorized_headlines[train_div:]
y_test = df['is_sarcastic'][train_div:]

print('The size of X_train is:', len(X_train), '\nThe size of y_train is:', len(y_train),
      '\nThe size of X_test is:', len(X_test), '\nThe size of y_test is:', len(y_test))

The size of X_train is: 20033 
The size of y_train is: 20033 
The size of X_test is: 8586 
The size of y_test is: 8586


In [30]:
## Need to reshape our data in order to convert it into the form expected by our CNN model

X_train = np.reshape(X_train, (len(X_train), MAX_LENGTH, VECTOR_SIZE))
X_test = np.reshape(X_test, (len(X_test), MAX_LENGTH, VECTOR_SIZE))
y_train = np.array(y_train)
y_test = np.array(y_test)

## Building the model

we are ready with our vectorized data, let's get into the CNN part. We begin by defining the hyperparameters of our network:

In [31]:
import keras
from keras.models import Sequential, Model
from keras import layers
from keras.layers import Dense, Dropout, Conv1D, GlobalMaxPooling1D

### Select the define hyperparameters

FILTERS= 10                                    ## The number of filters.
KERNEL_SIZE=3                                  ## indicates the number of tokens in the text
HIDDEN_LAYER_1_NODES= 20                       ## The number of nodes to be used in each of the hidden layers
HIDDEN_LAYER_2_NODES= 10            
DROPOUT_PROB=0.35                              ## indicating the percentage of nodes to be dropped at random.
NUM_EPOCHS=15                                  ## The number of epochs we'll see the entire data.
BATCH_SIZE=50                                  ## The number of vectorized headlines to input to the model in each batch.

Using TensorFlow backend.


In [32]:
model = Sequential()

model.add(Conv1D(FILTERS,
                 KERNEL_SIZE,
                 padding='same',
                 strides=1,                                       ## defined a stride of 1
                 activation='relu',                               ## ReLU as the activation function
                 input_shape = (MAX_LENGTH, VECTOR_SIZE)))
model.add(GlobalMaxPooling1D())                             ## used one-dimensional convolutions due to the single dimensionality associated with text data
model.add(Dense(HIDDEN_LAYER_1_NODES, activation='relu'))   ## define our feedforward neural network, along with the dropout layers
model.add(Dropout(DROPOUT_PROB))
model.add(Dense(HIDDEN_LAYER_2_NODES, activation='relu'))
model.add(Dropout(DROPOUT_PROB))

## Earlier layers, we used ReLU as the activation function and sigmoid in the last layer since we are trying to solve a binary classification problem
model.add(Dense(1, activation='sigmoid'))


In [33]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 10, 10)            9010      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                220       
_________________________________________________________________
dropout_1 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
_________________________________________________________________
dropout_2 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                

In [34]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [35]:
training_history = model.fit(X_train, y_train, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE)

Epoch 1/15
20033/20033 [==============================] - 3s 132us/step - loss: 0.6234 - accuracy: 0.6572
Epoch 2/15
20033/20033 [==============================] - 2s 99us/step - loss: 0.5256 - accuracy: 0.7520
Epoch 3/15
20033/20033 [==============================] - 2s 102us/step - loss: 0.4694 - accuracy: 0.7916
Epoch 4/15
20033/20033 [==============================] - 2s 101us/step - loss: 0.4342 - accuracy: 0.8135
Epoch 5/15
20033/20033 [==============================] - 2s 103us/step - loss: 0.4014 - accuracy: 0.8307
Epoch 6/15
20033/20033 [==============================] - 2s 103us/step - loss: 0.3777 - accuracy: 0.8425
Epoch 7/15
20033/20033 [==============================] - 2s 102us/step - loss: 0.3509 - accuracy: 0.8574
Epoch 8/15
20033/20033 [==============================] - 2s 103us/step - loss: 0.3268 - accuracy: 0.8675
Epoch 9/15
20033/20033 [==============================] - 2s 103us/step - loss: 0.3090 - accuracy: 0.8765
Epoch 10/15
20033/20033 [======================

In [36]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Testing Accuracy:  0.7469


**We get Training Accuracy as 90.93 %  and Test Accuracy as 74.69 %**

We can play with the Model Hyperparameters and observe the Training and Testing Accuracy, Focus on Testing Accuracy instead of training accuracy so that our Model should not Overfit the Data and leads to unexpected outcomes